# import

In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sys.path.append('../..')
import model.Baseline as baseline
import model.my_model as mymodel
import model.util_loss as util_ls
import model.util_dataloader as util_dl
import model.util_model as util_md
import data_process.util_data as util_dt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

# local
# file_save_dir = r'C:\Users\29492\Desktop\server_data\main_UKDALE'
# file_load_dir = r'D:\dataset\DataFrames\UKDALE'
# server
file_save_dir = r'../../results'
file_load_dir = r'../../data/UKDALE'

dm = util_md.DictManager(file_save_dir)
mwm = util_md.ModelWeightManager(file_save_dir)

# prepare

In [ ]:
dataframe_path_origin = r'house_1.csv'
house_origin_cols = [
 'boiler-2',
 'solar_thermal_pump-3',
 'laptop-4',
 'washing_machine-5',
 'dishwasher-6',
 'tv-7',
 'kitchen_lights-8',
 'htpc-9',
 'kettle-10',
 'toaster-11',
 'fridge-12',
 'microwave-13',
 'lcd_office-14',
 'hifi_office-15',
 'breadmaker-16',
 'amp_livingroom-17',
 'adsl_router-18',
 'livingroom_s_lamp-19',
 'soldering_iron-20',
 'gigE_&_USBhub-21',
 'hoover-22',
 'kitchen_dt_lamp-23',
 'bedroom_ds_lamp-24',
 'lighting_circuit-25',
 'livingroom_s_lamp2-26',
 'iPad_charger-27',
 'subwoofer_livingroom-28',
 'livingroom_lamp_tv-29',
 'DAB_radio_livingroom-30',
 'kitchen_lamp2-31',
 'kitchen_phone&stereo-32',
 'utilityrm_lamp-33',
 'samsung_charger-34',
 'bedroom_d_lamp-35',
 'coffee_machine-36',
 'kitchen_radio-37',
 'bedroom_chargers-38',
 'hair_dryer-39',
 'straighteners-40',
 'iron-41',
 'gas_oven-42',
 'data_logger_pc-43',
 'childs_table_lamp-44',
 'childs_ds_lamp-45',
 'baby_monitor_tx-46',
 'battery_charger-47',
 'office_lamp1-48',
 'office_lamp2-49',
 'office_lamp3-50',
 'office_pc-51',
 'office_fan-52',
 'LED_printer-53']


house_origin_cols_transfer = ['laptop-4', 'washing_machine-5', 'kettle-10', 'toaster-11', 'fridge-12', 'microwave-13']#
on_threshold = 30
dataframe_path_transfer = {r'house_2.csv':['laptop-2','washing_machine-12','kettle-8','toaster-16','fridge-14','microwave-15']}

'''

'''


min_val = 0
max_val = 7166
loss_factor=max_val-min_val

metric_dict_one2one = {'MAE(apps)':util_ls.MAE(single_class=True),
                'SAE':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=True)}

metric_dict_one2all = {'MAE(apps)':util_ls.MAE(single_class=False),
                       'MAE(mean)':util_ls.MAE(single_class=True),
                'SAE(apps)':util_ls.SignalAggregateError(single_class=False, period_len=450),
                'SAE(mean)':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=False)}

sliding_window_len = 599
in_seq_l = sliding_window_len

# DAE

In [ ]:
eval_b_s = 450*2
out_dim = 599

for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^aggregate').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        if not origin_app in ['fridge-12']:
            continue
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=sliding_window_len, offset=0)
        w_main = np.expand_dims(w_main, axis=1)
        w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.DAE(in_seq_l)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='UKDALE',model='DAE',appname=origin_app,date='0101'))

        score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                        return_metric_dict=metric_dict_one2one)

        del model, dataset, testset, w_main, w_apps, w_apps_on, score_test

# S2S

In [ ]:
eval_b_s = 450*2
out_dim = 599

for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^aggregate').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=sliding_window_len, offset=0)
        w_main = np.expand_dims(w_main, axis=1)
        # w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.Seq2Seq(in_seq_l, in_seq_l)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='UKDALE',model='S2S',appname=origin_app,date='0101'))

        score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                        return_metric_dict=metric_dict_one2one)

        del model, dataset, testset, w_main, w_apps, w_apps_on, score_test

# S2P

In [ ]:
eval_b_s = 450*4
out_dim = 1

for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^aggregate').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
        w_main = np.expand_dims(w_main, axis=1)
        w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.Seq2Point(in_seq_l, 1)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='UKDALE',model='S2P',appname=origin_app,date='1231'))

        score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                        return_metric_dict=metric_dict_one2one)

        del model, dataset, testset, w_main, w_apps, w_apps_on, score_test

# SGN

In [ ]:
eval_b_s = 450*10
out_dim = 1

for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^aggregate').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
        except KeyError:
            print(f'!!! {transfer_app} no found')
            continue

        w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=app, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
        w_main = np.expand_dims(w_main, axis=1)
        w_apps = np.expand_dims(w_apps, axis=1)
        w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))
        dataset.describe()
        testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

        model = baseline.SGN(in_seq_l)
        mwm.load_model_weight(model, '{dataset}_{model}_{appname}_{date}'
                              .format(dataset='UKDALE',model='SGN',appname=origin_app,date='0422'))

        score_test, _ = util_md.train_val_DoubleTask(device, model, valset=testset, loss_reg_factor=loss_factor,
                        return_metric_reg_dict=metric_dict_one2one)

        del model, dataset, testset, w_main, w_apps, w_apps_on, score_test

# MARK

In [ ]:
eval_b_s = 450*4
out_dim = len(house_origin_cols)


for (filename, cols) in dataframe_path_transfer.items():
    path = os.path.join(file_load_dir, filename)
    print('-'*40, f'file path:{path}', sep='\n')
    tmp_df = pd.read_csv(path)
    main = tmp_df.filter(regex='^aggregate').sum(axis=1).to_numpy()
    main = (main - min_val)/(max_val - min_val)
    apps = np.zeros((len(main), len(house_origin_cols)))

    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        print('-'*20)
        print(f'Matching A[{origin_app}] to B[{transfer_app}]')
        try:
            app = tmp_df[transfer_app]
            app = (app - min_val)/(max_val - min_val)
            print(f'--- {transfer_app} found')
            col_index = house_origin_cols.index(origin_app)
            apps[:,col_index] = app
        except KeyError:
            print(f'!!! {transfer_app} no found')

    w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
    w_main = np.expand_dims(w_main, axis=1)
    w_apps_sum = np.zeros((len(w_apps),1))
    w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
    dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                            torch.from_numpy(w_apps_on).to(torch.float32),
                            torch.from_numpy(w_apps_sum).to(torch.float32),
                            torch.from_numpy(w_apps).to(torch.float32))
    dataset.describe()
    testset = util_dl.just_to_DataLoader(dataset=dataset, batch_size=eval_b_s, shuffle_flag=False)

    model = mymodel.Model_final(in_seq_l, out_dim)
    mwm.load_model_weight(model,'{dataset}_{model}_{appname}_{date}'
                          .format(dataset='UKDALE',model='Final',appname='all',date=f'0101-{0}'))
    score_test, _ = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, valset=testset, loss_reg_factor=loss_factor,
                    return_metric_reg_dict=metric_dict_one2all)

    for origin_app, transfer_app in zip(house_origin_cols_transfer, cols):
        try:
            print(f'--- {transfer_app} found')
            col_index = house_origin_cols.index(origin_app)
            print(score_test['val']['MAE(apps)'][0, col_index])
            print(score_test['val']['SAE(apps)'][0, col_index])
            print(score_test['val']['MAE(offon)'][0][:, col_index])
        except KeyError:
            print(f'!!! {transfer_app} no found')

    del model, dataset, testset, w_main, w_apps, w_apps_on, score_test
